In [ ]:
import pandas as pd
import os

# Define the folder path where your CSV files are located
folder_path = 'Samples for WSP/SM/'

# Initialize an empty list to store the DataFrames
dataframes = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all the DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Create the output folder if it doesn't exist
output_folder = 'output'
os.makedirs(output_folder, exist_ok=True)

# Print the combined DataFrame
combined_df['Date'].unique()
combined_df

In [ ]:
# Iterate over each unique date in the "Date" column
for date in combined_df['Date'].unique():
    # Replace "/" with "-" in the date
    formatted_date = date.replace("/", "-")
    
    # Create a folder for the current date
    date_folder = os.path.join(output_folder, formatted_date)
    os.makedirs(date_folder, exist_ok=True)
    single_date_df = combined_df[combined_df['Date'] == date]

    # Convert 'Cycle_starttime' and 'Date' columns to datetime format
    single_date_df['Cycle start time'] = pd.to_datetime(single_date_df['Cycle start time'])

    single_date_df['Date'] = pd.to_datetime(single_date_df['Date'])

    # Round the 'Cycle_starttime' column to the nearest 5-minute interval
    single_date_df['Rounded_Time'] = single_date_df['Cycle start time'].dt.floor('5T')

    # Combine 'Date' and 'Rounded_Time' into a new column
    single_date_df['Combined_DateTime'] = single_date_df['Date'].astype(str) + ' ' + single_date_df['Rounded_Time'].dt.time.astype(str)

    # Rename the 'Rounded_Combined_DateTime' column to 'Combined_DateTime'
    single_date_df = single_date_df.rename(columns={'Rounded_Combined_DateTime': 'Combined_DateTime'})
    # Move the 'Combined_DateTime' column to be the 5th column
    cols = list(single_date_df.columns)
    cols.insert(4, cols.pop(cols.index('Combined_DateTime')))
    single_date_df = single_date_df[cols]
    
    for site_id in single_date_df['Site ID'].unique():
        site_id_df = single_date_df[single_date_df['Site ID'] == site_id]
        # Remove duplicate values in the 'Combined_DateTime' column
        site_id_df.drop_duplicates(subset='Combined_DateTime', inplace=True)
        # Drop Cycle_starttime column
        site_id_df = site_id_df.drop('Cycle start time', axis=1)
        single_date_df_folder = output_folder+"/"+formatted_date+f"/{site_id}.csv"
        # Write the DataFrame to a CSV file
        site_id_df.to_csv(single_date_df_folder, index=False)

single_date_df

In [ ]:

# Define the folder path where your CSV files are located
folder_path = 'Samples for WSP/VS/'

# Initialize an empty list to store the DataFrames
dataframes1 = []

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df1 = pd.read_csv(file_path)
        
        # Append the DataFrame to the list
        dataframes1.append(df1)

# Concatenate all the DataFrames into one
combined_df1 = pd.concat(dataframes1, ignore_index=True)

# # # Create the output folder if it doesn't exist
# output_folder = 'output'
# os.makedirs(output_folder, exist_ok=True)

# Print the combined DataFrame
combined_df1['Date'].unique()
combined_df1



In [ ]:
# Iterate over each unique date in the "Date" column
for date in combined_df1['Date'].unique():
    # Replace "/" with "-" in the date
    formatted_date1 = date.replace("/", "-")
        # # Create a folder for the current date
    # date_folder = os.path.join(output_folder, formatted_date)
    # os.makedirs(date_folder, exist_ok=True)
    TraffCountCHK7df = combined_df1[combined_df1['Date'] == date]
    # Convert 'Start_time' column in Traffic Count data to datetime format
    TraffCountCHK7df['Start time'] = pd.to_datetime(TraffCountCHK7df['Start time'])
            # Create a new column 'Combined_DateTime' by combining 'Date' and 'Start_time'
    TraffCountCHK7df['Combined_DateTime'] = TraffCountCHK7df['Date'].astype(str) + ' ' + TraffCountCHK7df['Start time'].dt.time.astype(str)
    # Convert 'Combined_DateTime' column to datetime format
    TraffCountCHK7df['Combined_DateTime'] = pd.to_datetime(TraffCountCHK7df['Combined_DateTime'])
    # Round 'Combined_DateTime' column down to the nearest 5 minutes
    TraffCountCHK7df['Rounded_Combined_DateTime'] = TraffCountCHK7df['Combined_DateTime'].dt.floor('5min')
    #Filter the rows in the 'TraffCount3df' DataFrame where the 'Site_ID' column that has the value "2368"
    for site_id in TraffCountCHK7df['Site ID'].unique():
        site_idfilt7_df = TraffCountCHK7df[TraffCountCHK7df['Site ID'] == site_id]
        # Pivot the DataFrame to create separate columns for each unique Detector_ID
        site_idpivot7_df = site_idfilt7_df.pivot(index=['Combined_DateTime', 'Site ID'], columns='Detector ID', values='Traffic volume')
        # Reset the index to make the combined DateTime and Site_ID columns regular columns
        site_idpivot7_df = site_idpivot7_df.reset_index()
        # Merge the pivoted DataFrame back into the original DataFrame based on the combined DateTime and Site_ID columns
        site_idmerged7_df = site_idfilt7_df.merge(site_idpivot7_df, on=['Combined_DateTime', 'Site ID'], how='left')
        # Remove the 'Combined_DateTime' column
        site_idmerged7_df.drop('Combined_DateTime', axis=1, inplace=True)
        # Rename the 'Rounded_Combined_DateTime' column to 'Combined_DateTime'
        site_idmerged7_df.rename(columns={'Rounded_Combined_DateTime': 'Combined_DateTime'}, inplace=True)
        # Drop the 'Detector_ID' column
        site_idmerged7_df.drop('Detector ID', axis=1, inplace=True)
        # Remove duplicate values in the 'Combined_DateTime' column
        site_idmerged7_df.drop_duplicates(subset='Combined_DateTime', inplace=True)
        # Drop Cycle_starttime column
        site_idmerged7_df = site_idmerged7_df.drop(['Start time', 'Traffic volume'], axis=1)
        # Convert 'Combined_DateTime' column to datetime format
        site_idmerged7_df['Combined_DateTime'] = pd.to_datetime(site_idmerged7_df['Combined_DateTime'])
        
        #single_date_df_folder = output_folder+"/"+formatted_date+f"/{site_idmerged7}.csv"
        # # Write the DataFrame to a CSV file
        #site_idmerged7_df.to_csv(single_date_df_folder, index=False)            
        # Print the updated DataFrame
        site_idmerged7_df
        
        



    

In [ ]:
# Convert datetime column to string in both dataframes
single_date_df['Combined_DateTime'] = single_date_df['Combined_DateTime'].astype(str)
site_idmerged7_df['Combined_DateTime'] = site_idmerged7_df['Combined_DateTime'].astype(str)

# Merge the single_date_df (Signal Phasing data) and the site_idmerged7_df (Traffic Volume data)

Newmerged9_df = single_date_df.merge(site_idmerged7_df, on='Combined_DateTime')
Newmerged9_df


# # Rename the dataframe
# new_df_name = f"site{site_id}_date{date_value}"
# renamed_df = Newmerged9_df.copy()
# renamed_df.name = new_df_name

# # Specify the path and filename for the CSV file
# Newmerged9 = f"C:/Users/nsdsc0/OneDrive - WSP O365/DATA_ANALYTICS/Python/SCATS/Output/str(data_value)/{site_id}.csv"

# # Write the DataFrame to a CSV file
# #Newmerged6_df.to_csv(Newmerged6, index=False)
# #Newmerged6_df

# # Save the renamed dataframe to a CSV file
# csv_filename = f"{new_df_name}.csv"
# renamed_df.to_csv(renamed, index=False)